<a href="https://colab.research.google.com/github/cmreyesvalencia-png/Assignment-AI-Projects/blob/main/A3_C13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 3: Machine Learning for Predicting Trading Signals**
- **Course:** Data Analytics and Business Intelligence Analyst
- **Institution:** Willis College
- **Student Name:** Carlos Reyes
- **Instructor:** Ratinder Rajpal
- **Date:** 2025 Nov, 22

# **Introduction**

- This assignment develop and evaluate supervised machine learning models to predict ’buy’ and ’sell’ signals for stocks, using manually computed technical indicators such as MACD, RSI, and Bollinger Bands. This project aims to apply advanced classification techniques to financial market data, facilitating effective trading decision-making.

- The assignment implement various machine learning models to generate predictive trading signals and evaluate these models based on their accuracy, precision, recall, and overall predictive power. Insights into the behavior of financial markets through technical analysis will also be gained.

In [ ]:
# Step 1 — Imports and setup
import os
import warnings
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import joblib
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

# Directories
INPUT_PARQUET = "clean_stock_data.parquet"
OUTPUT_DIR = "outputs"
MODEL_DIR = "models"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print("Libraries imported successfully!")


Libraries imported successfully!


In [ ]:
# Step 2 — Load Parquet
print("Loading cleaned data...")
df = pd.read_parquet(INPUT_PARQUET)
df = df.reset_index(drop=True)
print("Loaded data shape:", df.shape)

Loading cleaned data...
Loaded data shape: (16557818, 66)


# **1. Feature Engineering with Technical Indicators**

In this task we are utilizing domain knowledge for feature engineering. Manually compute MACD and RSI for each stock (By "manually," we refer to directly coding the formula using Python and Pandas, without relying on pre-built functions from external libraries or tools). Define buy, sell, and hold signals based on the behavior of these indicators. The signal is Buy if both indicators are showing Buy, and the signal is Sell if both indicators are showing Sell signal, otherwise the signal is to Hold.

## Moving Average Convergence Divergence (MACD)

**Definition:** MACD is calculated by subtracting the 26-period Exponential Moving Average (EMA) from the 12-period EMA. The result of this subtraction is the MACD line. A signal line, which is the 9-period EMA of the MACD line, is then plotted alongside the MACD line to act as a trigger for buy and sell signals.

- **MACD** = EMA₁₂(close) – EMA₂₆(close)
- **Signal Line** = EMA₉(MACD)

**Signals:**
- **Buy Signal:** When the MACD crosses above the signal line
- **Sell Signal:** When the MACD crosses below the signal line

## Relative Strength Index (RSI)

**Definition:** RSI is a momentum oscillator that measures the speed and change of price movements. RSI oscillates between zero and 100. Traditionally, RSI is considered overbought when above 70 and oversold when below 30.

- **RSI** = 100 - (100 ÷ (1 + RS))
- Where **RS (Relative Strength)** is the average gain divided by the average loss:
- **RS** = EMA(Gain) / EMA(Loss)
- These averages are typically calculated using an exponential moving average

**Signals:**
- **Buy Signal:** RSI levels below 30 suggest a potential upward turn
- **Sell Signal:** RSI levels above 70 suggest a potential downward turn

In [ ]:
# Step 3 — Indicator calculation functions
def compute_ema(series: pd.Series, span: int) -> pd.Series:
    """Compute exponential moving average."""
    return series.ewm(span=span, adjust=False).mean()

def compute_macd(df: pd.DataFrame, price_col: str = "close") -> pd.DataFrame:
    """Compute MACD and MACD signal per ticker."""
    df = df.copy()
    df["MACD"] = df.groupby("ticker")[price_col].transform(lambda s: compute_ema(s, 12) - compute_ema(s, 26))
    df["MACD_signal"] = df.groupby("ticker")["MACD"].transform(lambda s: compute_ema(s, 9))
    return df

def compute_rsi(df: pd.DataFrame, price_col: str = "close", period: int = 14) -> pd.DataFrame:
    """Compute RSI per ticker."""
    df = df.copy()
    def rsi_for_series(close: pd.Series) -> pd.Series:
        delta = close.diff()
        gain = delta.where(delta > 0, 0.0)
        loss = -delta.where(delta < 0, 0.0)
        avg_gain = gain.ewm(span=period, adjust=False).mean()
        avg_loss = loss.ewm(span=period, adjust=False).mean()
        rs = avg_gain / (avg_loss.replace(0, np.nan))
        rsi = 100 - (100 / (1 + rs))
        return rsi.fillna(50)
    df["RSI"] = df.groupby("ticker")[price_col].transform(rsi_for_series)
    return df

In [ ]:
# Step 4 — Generate signals
def generate_indicator_signals(df: pd.DataFrame) -> pd.DataFrame:
    """Generate trading signals using vectorized operations."""
    df = df.copy().sort_values(["ticker", "date"])

    print("Generating MACD signals...")
    df["MACD_prev"] = df.groupby("ticker")["MACD"].shift(1)
    df["MACD_signal_prev"] = df.groupby("ticker")["MACD_signal"].shift(1)

    conditions = [
        (df["MACD_prev"] < df["MACD_signal_prev"]) & (df["MACD"] > df["MACD_signal"]),
        (df["MACD_prev"] > df["MACD_signal_prev"]) & (df["MACD"] < df["MACD_signal"])
    ]
    choices = ["Buy", "Sell"]
    df["MACD_flag"] = np.select(conditions, choices, default="Neutral")

    print("Generating RSI signals...")
    conditions_rsi = [
        df["RSI"] < 30,
        df["RSI"] > 70
    ]
    choices_rsi = ["Buy", "Sell"]
    df["RSI_flag"] = np.select(conditions_rsi, choices_rsi, default="Neutral")

    print("Generating combined signals...")
    conditions_combined = [
        (df["MACD_flag"] == "Buy") & (df["RSI_flag"] == "Buy"),
        (df["MACD_flag"] == "Sell") & (df["RSI_flag"] == "Sell")
    ]
    choices_combined = ["Buy", "Sell"]
    df["signal"] = np.select(conditions_combined, choices_combined, default="Hold")

    df.drop(columns=["MACD_prev", "MACD_signal_prev"], inplace=True)
    return df

In [ ]:
# Compute technical indicators
print("Computing technical indicators...")
df = compute_macd(df)
df = compute_rsi(df)

# Generate signals
print("Generating trading signals...")
df = generate_indicator_signals(df)

print("Signal distribution:")
print(df['signal'].value_counts())
print(f"Buy signals: {(df['signal'] == 'Buy').sum() / len(df) * 100:.4f}%")
print(f"Sell signals: {(df['signal'] == 'Sell').sum() / len(df) * 100:.4f}%")

Computing technical indicators...
Generating trading signals...
Generating MACD signals...
Generating RSI signals...
Generating combined signals...
Signal distribution:
signal
Hold    16546033
Sell        5937
Buy         5848
Name: count, dtype: int64
Buy signals: 0.0353%
Sell signals: 0.0359%


# 2. Data Preparation and Splitting
- Integrate the indicators and the signals into the main dataset. Use the signals computed above as ground-truth labels for the dataset. Split the data into training and testing sets.


In [ ]:
# Step 5 — Prepare features and labels
def prepare_ml_dataset(df: pd.DataFrame):
    """Prepare dataset for machine learning."""
    df = df.copy().sort_values(["ticker", "date"])

    base_features = ["open","high","low","close","volume","ma_7","ma_30","volatility_30","daily_return","MACD","MACD_signal","RSI"]
    features = [c for c in base_features if c in df.columns]

    df["ticker_id"] = df["ticker"].astype("category").cat.codes
    features.append("ticker_id")

    if "signal" not in df.columns:
        raise KeyError("'signal' column not found.")

    df = df.dropna(subset=features + ["signal"]).copy()
    df["label"] = df["signal"].map({"Buy":1, "Hold":0, "Sell":2})

    X = df[features].copy()
    y = df["label"].copy()

    print(f"Final dataset - X shape: {X.shape}, y shape: {y.shape}")
    print("Label distribution:")
    print(y.value_counts().sort_index())

    return X, y, df

print("Preparing ML dataset...")
X, y, df_full = prepare_ml_dataset(df)

# Sample for memory efficiency
if len(X) > 1000000:
    print("Sampling to 1M records...")
    sample_idx = X.sample(n=1000000, random_state=42).index
    X = X.loc[sample_idx]
    y = y.loc[sample_idx]
    df_full = df_full.loc[sample_idx]

# Split data
print("Splitting data...")
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, shuffle=False, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.17647, shuffle=False, random_state=42)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

Preparing ML dataset...
Final dataset - X shape: (16557818, 13), y shape: (16557818,)
Label distribution:
label
0    16546033
1        5848
2        5937
Name: count, dtype: int64
Sampling to 1M records...
Splitting data...
Train: (700000, 13), Val: (150000, 13), Test: (150000, 13)


# 3. Model Building and Validation
- Implement 1-Logistic Regression, 2-Random Forests, 3-Support Vector Machines (SVM). Train the models on the training set and validate their performance. Training them can be time-consuming, depending on your computer's processing power.

In [ ]:
# Step 6 — ORIGINAL APPROACH (Baseline)
def original_approach(X_train, X_test, y_train, y_test):
    """Original training approach with basic class weighting."""
    print("\n" + "="*60)
    print("ORIGINAL APPROACH - Training Models")
    print("="*60)

    trained_models = {}
    results = []

    pipelines = {
        "LogisticRegression": (Pipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(max_iter=200, class_weight="balanced", random_state=42))
        ]), {
            "clf__C": [0.01, 0.1, 1.0]
        }),
        "RandomForest": (Pipeline([
            ("scaler", StandardScaler()),
            ("clf", RandomForestClassifier(class_weight="balanced", n_jobs=-1, random_state=42))
        ]), {
            "clf__n_estimators": [50, 100],
            "clf__max_depth": [5, 10, None]
        }),
        "SVM_Linear": (Pipeline([
            ("scaler", StandardScaler()),
            ("clf", SVC(kernel='linear', class_weight="balanced", cache_size=1000, random_state=42))
        ]), {
            "clf__C": [0.1, 1.0]
        })
    }

    skf = StratifiedKFold(n_splits=3, shuffle=False)

    for name, (pipe, params) in pipelines.items():
        print(f"\nTraining {name}...")

        if name == "SVM_Linear" and len(X_train) > 50000:
            print("Sampling 50,000 records for SVM training...")
            # CORRECTION: Use pandas sample instead of np.random.choice
            sample_indices = X_train.sample(n=50000, random_state=42).index
            X_train_model = X_train.loc[sample_indices]
            y_train_model = y_train.loc[sample_indices]
            svm_skf = StratifiedKFold(n_splits=2, shuffle=False)
            grid = GridSearchCV(pipe, params, cv=svm_skf, scoring="f1_weighted", n_jobs=-1)
            grid.fit(X_train_model, y_train_model)
        else:
            grid = GridSearchCV(pipe, params, cv=skf, scoring="f1_weighted", n_jobs=-1)
            grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        trained_models[name] = best_model

        y_pred = best_model.predict(X_test)

        # Basic metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="weighted", zero_division=0)
        rec = recall_score(y_test, y_pred, average="weighted", zero_division=0)
        f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)

        # Minority class metrics
        buy_recall = recall_score(y_test, y_pred, labels=[1], average=None)[0] if 1 in y_test.unique() else 0
        sell_recall = recall_score(y_test, y_pred, labels=[2], average=None)[0] if 2 in y_test.unique() else 0
        macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

        print(f"{name} — Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
        print(f"Buy Recall: {buy_recall:.4f}, Sell Recall: {sell_recall:.4f}, Macro F1: {macro_f1:.4f}")

        results.append({
            "model": name,
            "best_params": grid.best_params_,
            "accuracy": acc,
            "precision_weighted": prec,
            "recall_weighted": rec,
            "f1_weighted": f1,
            "macro_f1": macro_f1,
            "buy_recall": buy_recall,
            "sell_recall": sell_recall
        })

    return trained_models, results

In [ ]:
# Step 7 — OPTIMIZED ENHANCED APPROACH
def enhanced_approach_optimized(X_train, X_test, y_train, y_test):
    """OPTIMIZED Enhanced approach with faster training."""
    print("\n" + "="*60)
    print("OPTIMIZED ENHANCED APPROACH - Fast Training")
    print("="*60)

    trained_models = {}
    results = []

    def evaluate_model_enhanced(model, X_test, y_test, name):
        y_pred = model.predict(X_test)

        # Focus on key metrics only for speed
        acc = accuracy_score(y_test, y_pred)

        # Calculate minority class recall directly (faster)
        cm = confusion_matrix(y_test, y_pred)
        if cm.shape[0] >= 2:  # Ensure we have at least 2 classes
            buy_recall = cm[1,1] / cm[1,:].sum() if cm[1,:].sum() > 0 else 0
            sell_recall = cm[2,2] / cm[2,:].sum() if cm.shape[0] >= 3 and cm[2,:].sum() > 0 else 0
        else:
            buy_recall, sell_recall = 0, 0

        macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

        print(f"\n{name} Performance:")
        print(f"Accuracy: {acc:.4f}, Macro F1: {macro_f1:.4f}")
        print(f"Buy Recall: {buy_recall:.4f}, Sell Recall: {sell_recall:.4f}")
        print("Confusion Matrix:")
        print(cm)

        return {
            "accuracy": acc,
            "macro_f1": macro_f1,
            "buy_recall": buy_recall,
            "sell_recall": sell_recall,
            "confusion_matrix": cm
        }

    # OPTIMIZED SMOTE - Create smaller balanced dataset
    print("Applying OPTIMIZED SMOTE resampling...")
    print("Original class distribution:", y_train.value_counts().sort_index())

    # MAJOR OPTIMIZATION: Create smaller balanced dataset
    original_counts = y_train.value_counts()

    # Instead of oversampling to 699k each, create a manageable dataset
    # Target: 50k majority, 25k each minority (total ~100k samples vs 2.1M)
    target_majority = 50000
    target_minority = 25000

    print(f"Optimized sampling targets - Majority: {target_majority}, Minority: {target_minority}")

    # First, sample the majority class down
    majority_indices = X_train[y_train == 0].sample(n=target_majority, random_state=42).index
    minority_buy_indices = X_train[y_train == 1].index
    minority_sell_indices = X_train[y_train == 2].index

    # Combine sampled majority with all minority
    sampled_indices = list(majority_indices) + list(minority_buy_indices) + list(minority_sell_indices)
    X_train_sampled = X_train.loc[sampled_indices]
    y_train_sampled = y_train.loc[sampled_indices]

    # Now apply SMOTE only to boost minority classes
    smote = SMOTE(
        random_state=42,
        sampling_strategy={1: target_minority, 2: target_minority},
        k_neighbors=3  # Reduce neighbors for speed
    )

    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_sampled, y_train_sampled)
    print("Resampled class distribution:", pd.Series(y_train_resampled).value_counts().sort_index())
    print(f"Dataset size reduced from ~2.1M to {len(X_train_resampled):,} samples")

    # OPTIMIZED pipelines with faster hyperparameters
    pipelines = {
        "LogisticRegression_Enhanced": (ImbPipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(max_iter=200, class_weight="balanced", random_state=42))
        ]), {
            "clf__C": [0.1, 1.0]  # Reduced from 4 to 2 options
        }),

        "RandomForest_Enhanced": (ImbPipeline([
            ("scaler", StandardScaler()),
            ("clf", RandomForestClassifier(class_weight="balanced", n_jobs=-1, random_state=42))
        ]), {
            "clf__n_estimators": [100],  # Reduced from 2 to 1 option
            "clf__max_depth": [10, 20],  # Reduced from 3 to 2 options
            "clf__min_samples_split": [10],  # Reduced options
            "clf__min_samples_leaf": [2]     # Reduced options
        }),

        "SVM_Linear_Enhanced": (ImbPipeline([
            ("scaler", StandardScaler()),
            ("clf", SVC(kernel='linear', class_weight="balanced", cache_size=2000, random_state=42))
        ]), {
            "clf__C": [1.0]  # Single best parameter from previous runs
        })
    }

    for name, (pipe, params) in pipelines.items():
        print(f"\n{'='*50}")
        print(f"Training {name}...")
        print(f"{'='*50}")

        # OPTIMIZATION: Use fewer CV folds
        cv_strategy = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)  # Reduced from 3 to 2

        # OPTIMIZATION: Reduce verbosity and use faster scoring
        grid = GridSearchCV(
            pipe, params, cv=cv_strategy, scoring="f1_macro",
            n_jobs=-1, verbose=0  # Reduced verbosity
        )

        start_time = time.time()
        grid.fit(X_train_resampled, y_train_resampled)
        training_time = time.time() - start_time

        best_model = grid.best_estimator_
        trained_models[name] = best_model

        print(f"Training completed in {training_time:.1f} seconds")
        print(f"Best parameters: {grid.best_params_}")

        # Enhanced evaluation
        metrics = evaluate_model_enhanced(best_model, X_test, y_test, name)
        metrics.update({
            "model": name,
            "best_params": grid.best_params_,
            "training_time_seconds": training_time
        })
        results.append(metrics)

    return trained_models, results, X_train_resampled, y_train_resampled

In [ ]:
# Step 8 — Run Both Approaches (CORRECTED)
print("Starting model training...")

# Original Approach
print("\n" + "="*80)
print("RUNNING ORIGINAL APPROACH")
print("="*80)
original_models, original_results = original_approach(X_train, X_test, y_train, y_test)



Starting model training...

RUNNING ORIGINAL APPROACH

ORIGINAL APPROACH - Training Models

Training LogisticRegression...
LogisticRegression — Accuracy: 0.8768, Precision: 0.9994, Recall: 0.8768, F1: 0.9338
Buy Recall: 1.0000, Sell Recall: 1.0000, Macro F1: 0.3182

Training RandomForest...
RandomForest — Accuracy: 0.9994, Precision: 0.9988, Recall: 0.9994, F1: 0.9991
Buy Recall: 0.0000, Sell Recall: 0.0000, Macro F1: 0.3332

Training SVM_Linear...
Sampling 50,000 records for SVM training...
SVM_Linear — Accuracy: 0.8542, Precision: 0.9994, Recall: 0.8542, F1: 0.9207
Buy Recall: 1.0000, Sell Recall: 0.9750, Macro F1: 0.3129


In [ ]:
# Enhanced Approach - NOW USING OPTIMIZED VERSION
print("\n" + "="*80)
print("RUNNING OPTIMIZED ENHANCED APPROACH")
print("="*80)

enhanced_models, enhanced_results, X_train_resampled, y_train_resampled = enhanced_approach_optimized(X_train, X_test, y_train, y_test)


RUNNING OPTIMIZED ENHANCED APPROACH

OPTIMIZED ENHANCED APPROACH - Fast Training
Applying OPTIMIZED SMOTE resampling...
Original class distribution: label
0    699508
1       226
2       266
Name: count, dtype: int64
Optimized sampling targets - Majority: 50000, Minority: 25000
Resampled class distribution: label
0    50000
1    25000
2    25000
Name: count, dtype: int64
Dataset size reduced from ~2.1M to 100,000 samples

Training LogisticRegression_Enhanced...
Training completed in 7.6 seconds
Best parameters: {'clf__C': 1.0}

LogisticRegression_Enhanced Performance:
Accuracy: 0.8820, Macro F1: 0.3195
Buy Recall: 1.0000, Sell Recall: 1.0000
Confusion Matrix:
[[132213   7942   9752]
 [     0     53      0]
 [     0      0     40]]

Training RandomForest_Enhanced...
Training completed in 28.3 seconds
Best parameters: {'clf__max_depth': 20, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 10, 'clf__n_estimators': 100}

RandomForest_Enhanced Performance:
Accuracy: 0.9701, Macro F1: 

# **4. Model Evaluation and Optimization**
- Evaluate the models on the test set. Optimize the models based on evaluation metrics and adjust hyperparameters as needed to improve performance. Use cross-validation where applicable to ensure the robustness of the evaluation.

In [ ]:
# Step 9 — BASELINE vs OPTIMIZED PERFORMANCE COMPARISON
def compare_approaches(baseline_results, optimized_results):
    """Compare Baseline vs Optimized Enhanced approach performance"""
    print("\n" + "="*80)
    print("PERFORMANCE COMPARISON: BASELINE vs OPTIMIZED ENHANCED")
    print("="*80)

    comparison_data = []

    print("📊 DIRECT PERFORMANCE COMPARISON")
    print("-" * 60)

    # Compare each model type
    for base_model in ['LogisticRegression', 'RandomForest', 'SVM_Linear']:
        # Find baseline results
        baseline_model = next((r for r in baseline_results if base_model in r['model']), None)
        # Find optimized results
        optimized_model = next((r for r in optimized_results if base_model in r['model']), None)

        if baseline_model and optimized_model:
            # Calculate changes
            accuracy_change = optimized_model['accuracy'] - baseline_model['accuracy']
            buy_recall_change = optimized_model['buy_recall'] - baseline_model['buy_recall']
            sell_recall_change = optimized_model['sell_recall'] - baseline_model['sell_recall']
            macro_f1_change = optimized_model['macro_f1'] - baseline_model['macro_f1']

            # Display comparison
            model_name = base_model.replace('_', ' ')
            print(f"\n{model_name}:")
            print(f"  Accuracy:    {baseline_model['accuracy']:.4f} → {optimized_model['accuracy']:.4f} ({accuracy_change*100:+.2f}%)")
            print(f"  Buy Recall:  {baseline_model['buy_recall']:.4f} → {optimized_model['buy_recall']:.4f} ({buy_recall_change*100:+.2f}%)")
            print(f"  Sell Recall: {baseline_model['sell_recall']:.4f} → {optimized_model['sell_recall']:.4f} ({sell_recall_change*100:+.2f}%)")
            print(f"  Macro F1:    {baseline_model['macro_f1']:.4f} → {optimized_model['macro_f1']:.4f} ({macro_f1_change*100:+.2f}%)")

            # Store comparison
            comparison_data.append({
                'Model': model_name,
                'Accuracy_Baseline': baseline_model['accuracy'],
                'Accuracy_Optimized': optimized_model['accuracy'],
                'Buy_Recall_Baseline': baseline_model['buy_recall'],
                'Buy_Recall_Optimized': optimized_model['buy_recall'],
                'Sell_Recall_Baseline': baseline_model['sell_recall'],
                'Sell_Recall_Optimized': optimized_model['sell_recall'],
                'Macro_F1_Baseline': baseline_model['macro_f1'],
                'Macro_F1_Optimized': optimized_model['macro_f1']
            })

    # Create comparison DataFrame
    comparison_df = pd.DataFrame(comparison_data)

    # Summary statistics
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)

    if not comparison_df.empty:
        accuracy_improvement = (comparison_df['Accuracy_Optimized'] - comparison_df['Accuracy_Baseline']).mean()
        buy_recall_improvement = (comparison_df['Buy_Recall_Optimized'] - comparison_df['Buy_Recall_Baseline']).mean()
        sell_recall_improvement = (comparison_df['Sell_Recall_Optimized'] - comparison_df['Sell_Recall_Baseline']).mean()
        macro_f1_improvement = (comparison_df['Macro_F1_Optimized'] - comparison_df['Macro_F1_Baseline']).mean()

        print(f"Average Accuracy Change:    {accuracy_improvement*100:+.2f}%")
        print(f"Average Buy Recall Change:  {buy_recall_improvement*100:+.2f}%")
        print(f"Average Sell Recall Change: {sell_recall_improvement*100:+.2f}%")
        print(f"Average Macro F1 Change:    {macro_f1_improvement*100:+.2f}%")

    return comparison_df

# Run comparison
comparison_df = compare_approaches(original_results, enhanced_results)


PERFORMANCE COMPARISON: BASELINE vs OPTIMIZED ENHANCED
📊 DIRECT PERFORMANCE COMPARISON
------------------------------------------------------------

LogisticRegression:
  Accuracy:    0.8768 → 0.8820 (+0.52%)
  Buy Recall:  1.0000 → 1.0000 (+0.00%)
  Sell Recall: 1.0000 → 1.0000 (+0.00%)
  Macro F1:    0.3182 → 0.3195 (+0.13%)

RandomForest:
  Accuracy:    0.9994 → 0.9701 (-2.93%)
  Buy Recall:  0.0000 → 0.8113 (+81.13%)
  Sell Recall: 0.0000 → 0.7500 (+75.00%)
  Macro F1:    0.3332 → 0.3503 (+1.71%)

SVM Linear:
  Accuracy:    0.8542 → 0.8791 (+2.50%)
  Buy Recall:  1.0000 → 1.0000 (+0.00%)
  Sell Recall: 0.9750 → 1.0000 (+2.50%)
  Macro F1:    0.3129 → 0.3188 (+0.59%)

SUMMARY STATISTICS
Average Accuracy Change:    +0.03%
Average Buy Recall Change:  +27.04%
Average Sell Recall Change: +25.83%
Average Macro F1 Change:    +0.81%


In [ ]:
# Step 10 — Save models and predictions
print("Saving models and predictions...")

# Save enhanced models to MODEL_DIR
for model_name, model in enhanced_models.items():
    model_path = os.path.join(MODEL_DIR, f"{model_name.lower()}_model.joblib")
    joblib.dump(model, model_path)
    print(f"Saved {model_name} model to {model_path}")

# Save predictions to OUTPUT_DIR
for model_name, model in enhanced_models.items():
    # Create predictions DataFrame
    df_pred = df_full.loc[X_test.index, ["date", "ticker", "close", "signal"]].copy()
    df_pred[f"pred_{model_name}"] = model.predict(X_test)

    # Save predictions as Parquet
    pred_path = os.path.join(OUTPUT_DIR, f"predictions_{model_name}.parquet")
    df_pred.to_parquet(pred_path, index=False)
    print(f"Saved {model_name} predictions to {pred_path}")

# Save results to OUTPUT_DIR (remove problematic columns first)
def clean_results(results):
    """Remove columns that can't be saved to Parquet"""
    cleaned = []
    for result in results:
        clean_result = result.copy()
        # Remove problematic columns
        if 'confusion_matrix' in clean_result:
            del clean_result['confusion_matrix']
        if 'best_params' in clean_result:
            # Convert dict to string if needed
            if isinstance(clean_result['best_params'], dict):
                clean_result['best_params'] = str(clean_result['best_params'])
        cleaned.append(clean_result)
    return cleaned

# Clean and save results
baseline_clean = clean_results(original_results)
baseline_df = pd.DataFrame(baseline_clean)
baseline_df.to_parquet(os.path.join(OUTPUT_DIR, "baseline_results.parquet"), index=False)

enhanced_clean = clean_results(enhanced_results)
enhanced_df = pd.DataFrame(enhanced_clean)
enhanced_df.to_parquet(os.path.join(OUTPUT_DIR, "enhanced_results.parquet"), index=False)

comparison_df.to_parquet(os.path.join(OUTPUT_DIR, "comparison_results.parquet"), index=False)

print("All models, predictions and results saved successfully!")
print(f"Models saved in: {MODEL_DIR}")
print(f"Predictions and results saved in: {OUTPUT_DIR}")

Saving models and predictions...
Saved LogisticRegression_Enhanced model to models\logisticregression_enhanced_model.joblib
Saved RandomForest_Enhanced model to models\randomforest_enhanced_model.joblib
Saved SVM_Linear_Enhanced model to models\svm_linear_enhanced_model.joblib
Saved LogisticRegression_Enhanced predictions to outputs\predictions_LogisticRegression_Enhanced.parquet
Saved RandomForest_Enhanced predictions to outputs\predictions_RandomForest_Enhanced.parquet
Saved SVM_Linear_Enhanced predictions to outputs\predictions_SVM_Linear_Enhanced.parquet
All models, predictions and results saved successfully!
Models saved in: models
Predictions and results saved in: outputs
